In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [4]:
df = pd.read_excel('../../data/globalterrorismdb_0522dist.xlsx')


In [5]:
pd.set_option('display.max_columns', None)
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaT,58,Dominican Republic,2,Central America & Caribbean,National,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaT,130,Mexico,1,North America,Federal,Mexico city,19.371887,-99.086624,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",21.0,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23rd of September Communist League,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaT,160,Philippines,5,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaT,78,Greece,8,Western Europe,Attica,Athens,37.997490,23.762728,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Embassy,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,16.0,Unknown Explosive Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosive,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Na

In [6]:
def get_boolean_like_columns(df):
    boolean_columns = []
    
    for col in df.select_dtypes(include=['number']).columns:
        unique_values = df[col].dropna().unique()
        if set(unique_values).issubset({0, 1, 0.0, 1.0}):
            boolean_columns.append(col)
    
    return boolean_columns

# Récupération des colonnes booléennes
boolean_columns = get_boolean_like_columns(df)
print("Colonnes booléennes probables :", boolean_columns)

Colonnes booléennes probables : ['extended', 'crit1', 'crit2', 'crit3', 'multiple', 'success', 'suicide', 'guncertain1', 'guncertain2', 'guncertain3', 'individual', 'claim3']


In [7]:
from mlxtend.frequent_patterns import apriori, association_rules
df_bool = df[boolean_columns]
df_bool.head()

# remplacement des NaN par 0
df_bool.fillna(0, inplace=True)
supports = apriori(df_bool, min_support=0.1, use_colnames=True)

# Calculer les règles d'association (optionnel)
rules = association_rules(supports, metric="lift", min_threshold=1)

supports.sort_values(by="support", ascending=False).head(10)



C:\Users\benva\AppData\Local\Temp\ipykernel_18452\1508172267.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bool.fillna(0, inplace=True)
c:\Users\benva\.conda\envs\cuda\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
1,0.993291,(crit2)
0,0.988265,(crit1)
5,0.981555,"(crit2, crit1)"
4,0.883628,(success)
11,0.877338,"(crit2, success)"
8,0.872893,"(success, crit1)"
17,0.866604,"(crit2, success, crit1)"
2,0.866246,(crit3)
9,0.859537,"(crit2, crit3)"
6,0.854511,"(crit3, crit1)"


In [8]:
supports["len"] = supports["itemsets"].apply(lambda x: len(x))
pd.set_option("display.max_colwidth", None)
supports[supports["len"]==2].sort_values(by="support", ascending=False).head(10)

,support,itemsets,len
5,0.981555,"(crit2, crit1)",2
11,0.877338,"(crit2, success)",2
8,0.872893,"(success, crit1)",2
9,0.859537,"(crit2, crit3)",2
6,0.854511,"(crit3, crit1)",2
13,0.765972,"(success, crit3)",2
7,0.145380,"(multiple, crit1)",2
10,0.145241,"(crit2, multiple)",2
14,0.131041,"(success, multiple)",2
12,0.130597,"(multiple, crit3)",2


In [9]:
# compute association rules
rules = association_rules(supports, min_threshold=0.5)
rules.head(50)

# filtrer par lift descroissant
rules.sort_values(by="lift", ascending=True).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
72,"(crit3, success)","(crit2, crit1)",0.765972,0.981555,0.748949,0.977775,0.996149,-0.002896,0.829911,-0.016252
71,"(crit2, crit1)","(crit3, success)",0.981555,0.765972,0.748949,0.763022,0.996149,-0.002896,0.987552,-0.173286
20,(crit3),"(crit2, crit1)",0.866246,0.981555,0.847801,0.978707,0.997098,-0.002467,0.866246,-0.021293
17,"(crit2, crit1)",(crit3),0.981555,0.866246,0.847801,0.863733,0.997098,-0.002467,0.981555,-0.136267
78,(crit1),"(crit2, crit3, success)",0.988265,0.759683,0.748949,0.757842,0.997577,-0.001819,0.992400,-0.171457
65,"(crit2, crit3, success)",(crit1),0.759683,0.988265,0.748949,0.985870,0.997577,-0.001819,0.830555,-0.010004
77,(crit3),"(crit2, success, crit1)",0.866246,0.866604,0.748949,0.864591,0.997677,-0.001744,0.985136,-0.017107
66,"(crit2, success, crit1)",(crit3),0.866604,0.866246,0.748949,0.864234,0.997677,-0.001744,0.985181,-0.017152
34,"(crit3, success)",(crit1),0.765972,0.988265,0.755238,0.985986,0.997695,-0.001745,0.837431,-0.009777
39,(crit1),"(crit3, success)",0.988265,0.765972,0.755238,0.764207,0.997695,-0.001745,0.992511,-0.164499


In [10]:
df2 = pd.read_csv('../../data/processed/db_2021_preprocessed.csv')
df2.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../data/processed/db_2021_preprocessed.csv'

In [ ]:
def get_boolean_like_columns(df):
    boolean_columns = []
    
    for col in df.select_dtypes(include=['number']).columns:
        unique_values = df[col].dropna().unique()
        if set(unique_values).issubset({0, 1, 0.0, 1.0}):
            boolean_columns.append(col)
    
    return boolean_columns

# Récupération des colonnes booléennes
boolean_columns = get_boolean_like_columns(df2)
print("Colonnes booléennes probables :", boolean_columns)

Colonnes booléennes probables : ['vicinity', 'crit1', 'crit2', 'crit3', 'doubtterr', 'multiple', 'success', 'suicide', 'gsubname3', 'guncertain1', 'guncertain2', 'guncertain3', 'individual', 'claimed', 'claim2', 'claim3', 'compclaim', 'weapsubtype4_txt', 'INT_MISC', 'weapon_biological', 'weapon_chemical', 'weapon_radiological', 'weapon_nuclear', 'weapon_firearms', 'weapon_explosives', 'weapon_fake weapons', 'weapon_incendiary', 'weapon_melee', 'weapon_vehicle', 'weapon_sabotage equipment', 'weapon_other', 'weapon_unknown', 'attack_assassination', 'attack_armed assault', 'attack_bombing/explosion', 'attack_hijacking', 'attack_hostage taking (barricade incident)', 'attack_hostage taking (kidnapping)', 'attack_facility/infrastructure attack', 'attack_unarmed assault', 'attack_unknown', 'target_business', 'target_government (general)', 'target_police', 'target_military', 'target_abortion related', 'target_airports & aircraft', 'target_government (diplomatic)', 'target_educational instituti

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
df_bool = df2[boolean_columns]
df_bool.head()




,vicinity,crit1,crit2,crit3,doubtterr,multiple,success,suicide,gsubname3,guncertain1,guncertain2,guncertain3,individual,claimed,claim2,claim3,compclaim,weapsubtype4_txt,INT_MISC,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown,target_business,target_government (general),target_police,target_military,target_abortion related,target_airports & aircraft,target_government (diplomatic),target_educational institution,target_food or water supply,target_journalists & media,target_maritime,target_ngo,target_other,target_private citizens & property,target_religious figures/institutions,target_telecommunication
0,0,1,1,0,1,0.0,1,0,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,0.0,1,0,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0.0,1,0,NaN,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,1,1,1,1,0.0,1,0,NaN,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,1,1,1,0,0.0,1,0,NaN,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
supports = apriori(df_bool, min_support=0.1, use_colnames=True)

# Calculer les règles d'association (optionnel)
rules = association_rules(supports, metric="lift", min_threshold=1)

supports.sort_values(by="support", ascending=False).head(10)


C:\Users\emili\AppData\Local\Temp\ipykernel_26600\882931880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bool.fillna(0, inplace=True)
c:\Users\emili\anaconda3\envs\dm\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
1,0.996976,(crit2)
0,0.985081,(crit1)
19,0.982056,"(crit2, crit1)"
5,0.851613,(success)
40,0.848992,"(crit2, success)"
23,0.838911,"(crit1, success)"
93,0.836290,"(crit2, crit1, success)"
2,0.785484,(crit3)
37,0.782460,"(crit2, crit3)"
20,0.770565,"(crit1, crit3)"
